<a href="https://colab.research.google.com/github/SidIzG/DataScience/blob/master/Twitter_Sentiment_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tweepy

In [2]:
pip install -U textblob

     |████████████████████████████████| 636 kB 33.4 MB/s 
  Attempting uninstall: textblob
    Found existing installation: textblob 0.15.3
    Uninstalling textblob-0.15.3:
      Successfully uninstalled textblob-0.15.3


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import re
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import tweepy
import datetime
import time
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
stop_words = stopwords.words('english')

In [5]:
#CONSUMER_KEY='wPocpBJ1SBIX72ChrmNVmwqW7'
#CONSUMER_SECRET='19YXTSdjPDTerzmucR6trlfjcWLyHUNDRHIgrpa47SljJSVC17'
#ACCESS_KEY='1213222796219707392-Llpv2CHK5chB7wMS8wZO3JkZ4ag2Tp'
#ACCESS_SECRET='r9TjvQfBtboe6KXvFuxpxX1MVwsujIMgfBreZWjgOJ8DC'

CONSUMER_KEY='HFHkPO52n5vTc1QV3lcoIlY5u'
CONSUMER_SECRET='erzvmAHv1ALrrmBSNzwdDr9BkUBnQ9WJfVTtlcwSnLGI12LrBw'
ACCESS_KEY='815789471966720000-0F5talxU1n78Yd8mgO9TfWzXawgp9Mw'
ACCESS_SECRET='cuEyGoG0JtBDZGf6XVrFVXDVpgZZn8XS6OuxE3aL0YwKc'
#Bearer_Token = 'AAAAAAAAAAAAAAAAAAAAAK5pUwEAAAAACNS8TM1F7HVMzgRMJidXuMnb69M%3DMJfx9dRznYa3v1BAiSYLGT38TcJhNHq8m7epWfrLKl8o0GGaPS'
auth=tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY,ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [6]:
#Left leaning independent media sources
#https://libguides.ithaca.edu/indy/indy
#TYT, The Humanist Report, Justice Democrats, Rebel HQ, Majority Report, The Ring of Fire, Wolf-PAC, The Intercept, The Damage Report, The Hill, Democracy Now!
#Secular talk(@KyleKulinski), Krystal Ball (@krystalball), David Pakman (@dpakman), Corin's world (@cornchipsss), The Progressive Voice (@progressvoice), Emma Vigeland (@EmmaVigeland), Ana Kasparian (@AnaKasparian), Sam Seder (@SamSeder), Cenk Uygur (@cenkuygur), John Iadarola (@johniadarola), Jimmy Dore (@jimmy_dore)
#Popular left wing politicians: Bernie Sanders, Tulsi Gabbard, Nina Turner, Alexandria Ocasio Cortez, Ilhan Omar
#Right leaning independent media sources
#https://www.thoughtco.com/the-top-conservative-news-and-opinion-websites-3303614
# Turning Point USA, Daily Caller, Judicial Watch, Newsmax, The Federalist, Townhall.com, RedState, Heritage Foundation, TheBlaze, Conservative Review, Washington Free Beacon  
# Charlie Kirk (@charliekirk11), Candace Owens (@RealCandaceO), Dave Rubin (@RubinReport), Glenn Beck (@glennbeck), Ben Shapiro (@benshapiro), Steven Crowder (@scrowder), Dinesh D'souza (@DineshDSouza), Hodgetwins (@hodgetwins), Michael Knowles (@michaeljknowles), Matt Walsh (@MattWalshBlog), Jesse Lee Peterson (@JLPtalk)


In [7]:
left_wing_journalists = {"Secular talk":"@KyleKulinski", "Krystal Ball":"@krystalball", "David Pakman":"@dpakman", "Corin's world":"@cornchipsss", "The Progressive Voice":"@progressvoice", "Emma Vigeland":"@EmmaVigeland", "Ana Kasparian":"@AnaKasparian", "Sam Seder":"@SamSeder", "Cenk Uygur":"@cenkuygur", "John Iadarola":"@johniadarola", "Jimmy Dore":"@jimmy_dore"}#, "Hasan Abi":"@hasanthehun"}
right_wing_journalists = {"Charlie Kirk":"@charliekirk11", "Candace Owens":"@RealCandaceO", "Dave Rubin":"@RubinReport", "Glenn Beck":"@glennbeck", "Ben Shapiro":"@benshapiro", "Steven Crowder":"@scrowder", "Dinesh D'souza":"@DineshDSouza", "Hodgetwins":"@hodgetwins", "Michael Knowles":"@michaeljknowles", "Matt Walsh":"@MattWalshBlog", "Jesse Lee Peterson":"@JLPtalk"}#, "Tomi Lahren":"@TomiLahren"}
left_wing_independent = {"TYT":"@TYT", "The Humanist Report":"@HumanistReport", "Justice Democrats":"@justicedems", "Rebel HQ":"@TYTPolitics", "Majority Report":"@majorityfm", "The Ring of Fire":"@RingOfFireMedia", "Wolf-PAC":"@WolfPAChq", "The Intercept":"@theintercept", "The Damage Report":"@TheDamageReport", "The Hill":"@thehill", "Democracy Now!":"@democracynow"} #, "The Democrats":"@TheDemocrats"}
right_wing_independent = {"Turning Point USA":"@TPUSA", "Daily Caller":"@DailyCaller", "Judicial Watch":"@JudicialWatch", "Newsmax":"@newsmax", "The Federalist":"@FDRLST", "Townhall.com":"@townhallcom", "RedState":"@RedState", "Heritage Foundation":"@Heritage", "TheBlaze":"@theblaze", "Conservative Review":"@CR", "Washington Free Beacon":"@FreeBeacon"}#, "GOP":"@GOP"}

In [8]:
#news_topics = ["Covid-19", "Coronavirus", "Infrastructure Bill", "Police Vaccine Mandate", "Pfizer vaccine", "Taiwan", "Climate Change", "US Strikes", "School Mask Mandate", "Afghanistan", "Capitol Riots", "Socialism", "Universal Healthcare", "Gun control"]

In [9]:
import requests

url = "https://google-news.p.rapidapi.com/v1/topic_headlines"

querystring = {"lang":"en","country":"US","topic":"nation"}

headers = {
    'x-rapidapi-host': "google-news.p.rapidapi.com",
    'x-rapidapi-key': "da2c1768femshc74ec1e1c989e92p170d02jsnc7ddcd0e1879"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
data = response.json()

In [125]:
data["articles"]

[{'id': 'CBMifWh0dHBzOi8vYXBuZXdzLmNvbS9hcnRpY2xlL3N0ZXZlLWJhbm5vbi1kb25hbGQtdHJ1bXAtYmFyYWNrLW9iYW1hLWltcGVhY2htZW50cy1jYXBpdG9sLXNpZWdlLTZmYjI0YTM4YTQxOWVmZGM2NmJlM2EwMzg4ZTlmYmNj0gEA',
  'link': 'https://news.google.com/__i/rss/rd/articles/CBMifWh0dHBzOi8vYXBuZXdzLmNvbS9hcnRpY2xlL3N0ZXZlLWJhbm5vbi1kb25hbGQtdHJ1bXAtYmFyYWNrLW9iYW1hLWltcGVhY2htZW50cy1jYXBpdG9sLXNpZWdlLTZmYjI0YTM4YTQxOWVmZGM2NmJlM2EwMzg4ZTlmYmNj0gEA?oc=5',
  'published': 'Sat, 13 Nov 2021 05:03:31 GMT',
  'source': {'href': 'https://apnews.com', 'title': 'Associated Press'},
  'sub_articles': [{'publisher': 'Associated Press',
    'title': 'Bannon indicted on contempt charges for defying 1/6 subpoena',
    'url': 'https://news.google.com/__i/rss/rd/articles/CBMifWh0dHBzOi8vYXBuZXdzLmNvbS9hcnRpY2xlL3N0ZXZlLWJhbm5vbi1kb25hbGQtdHJ1bXAtYmFyYWNrLW9iYW1hLWltcGVhY2htZW50cy1jYXBpdG9sLXNpZWdlLTZmYjI0YTM4YTQxOWVmZGM2NmJlM2EwMzg4ZTlmYmNj0gEA?oc=5'},
   {'publisher': 'CBS Mornings',
    'title': 'Former Trump adviser Steve Bannon 

In [ ]:
#instead of independent media source, we will use the media sources from the api. 

In [31]:
def get_topics(headlines):
  stop_words = stopwords.words('english')

  tokens=[token.strip() \
  for token in nltk.word_tokenize(headlines.lower()) \
      if token.strip() not in stop_words and \
        token.strip() not in string.punctuation]
  tokens=[token.strip() for token in tokens \
        if token.strip()!=' ']
  tokens=[token.strip() for token in tokens \
        if token.strip()!="'s"]

  lmtzr = nltk.WordNetLemmatizer()
  lemmatized_tokens = [lmtzr.lemmatize(token) for token in tokens]
  trigrams=list(nltk.bigrams(lemmatized_tokens))
  token_count=nltk.FreqDist(trigrams)
  phrases_dict = dict(token_count)
  sorted_phrases_dict = dict(sorted(phrases_dict.items(), key = lambda x: x[1], reverse = True))
  list_sorted_phrases = list(sorted_phrases_dict.items())
  topic =" ".join(list_sorted_phrases[0][0])
  return topic

In [ ]:
#can use 2 datasets. one for bigrams and one for trigrams

In [32]:
topics = []
for x in range(len(data["articles"])):
  headlines = []
  for i in range(len(data["articles"][x]["sub_articles"])):
    headlines.append(data["articles"][x]["sub_articles"][i]['title'])
    tokenization_headlines = ",".join(headlines)
  topics.append(get_topics(tokenization_headlines))

In [33]:
topics

['steve bannon',
 'ahmaud arbery',
 'cop26 climate',
 'kyle rittenhouse',
 'homer plessy',
 'capitol riot',
 'capitol riot',
 'vaccine mandate',
 'plane crash',
 '2 high',
 'central park',
 'booster shot',
 'sarah sander',
 'restraining order',
 'missing 5-year-old',
 'declares state',
 'declares state',
 'national guard',
 '24 year',
 'u.s. capitol',
 'd.c.-area forecast',
 'found safe',
 'bank robber',
 'found safe',
 'rpd man',
 'amber alert',
 'black autistic',
 'teacher found',
 "'hang mike",
 'coronavirus oregon',
 'water level',
 'university chicago',
 'snow accumulation',
 'murkowski announces',
 'fog expected',
 'isabella kalua',
 'gov bill',
 'young wheeling',
 'tracking saturday',
 'ciattarelli concedes',
 'andrew cuomo',
 'andrew cuomo',
 'new covid',
 'john deere',
 'john deere',
 '2 adult',
 'robert califf',
 'elizabeth holmes',
 'school board',
 'golden state',
 "'dolphin tale",
 'madison cawthorn',
 'chris christie',
 'joe biden',
 'cheney rip',
 'hit-and-run outside',


In [13]:
left_wing_handles = list(left_wing_journalists.values())
right_wing_handles = list(right_wing_journalists.values())
right_wing_handles.extend(list(right_wing_independent.values()))
left_wing_handles.extend(list(left_wing_independent.values()))
news_handles = left_wing_handles[:]
news_handles.extend(right_wing_handles)

In [14]:
twitter_data_df = pd.DataFrame(left_wing_handles, columns=["News Twitter Handles"])

In [15]:
names_of_users = []
for handle in news_handles:
  names_of_users.append((api.get_user(handle)).name)

In [27]:
#add leftwing or rightwing column by searching recommended users

In [17]:
twitter_data_df[topics] = None

In [126]:
# initialize the first call
alltweets=[]
tweet_strings = []
new_tweets=api.user_timeline(screen_name="@washingtonpost", 
                           # 200 is the maximum allowed count
                           count=200,
                           #include_rts = False,
                           tweet_mode = 'extended')
alltweets.extend(new_tweets)
oldest=alltweets[-1].id-1  #next time start from the oldest one minus one 
    
    # continue to get tweets
while len(new_tweets)>0:  
    print ("getting tweets before", oldest)
    new_tweets = api.user_timeline(screen_name="@KyleKulinski",count=200, tweet_mode='extended', max_id=oldest)#include_rts=False
    alltweets.extend(new_tweets)
    oldest=alltweets[-1].id-1

for i in range(len(alltweets)):
  temp_str = alltweets[i].full_text
  tweet_strings.append(temp_str)
#outtweets = [[tweet.id_str, tweet.user.name, tweet.created_at, tweet.user.followers_count, tweet.text.encode("utf-8")] for tweet in alltweets]

getting tweets before 1458892139782447107
getting tweets before 1453495667586912257
getting tweets before 1445079144673460240
getting tweets before 1437766655627628547
getting tweets before 1427381902068097031
getting tweets before 1412497536192069632
getting tweets before 1404914762232872959
getting tweets before 1391950628369358847
getting tweets before 1370547147154845696
getting tweets before 1354446683405430786
getting tweets before 1351366954754461701
getting tweets before 1341490351262068738
getting tweets before 1336822748501463039
getting tweets before 1330344334462504959
getting tweets before 1325250971103014913
getting tweets before 1320929121820696577
getting tweets before 1319441207278424069
getting tweets before 1318388884003315711


In [127]:
filtered_tweets = []

for i in range(len(tweet_strings)):
  if "vaccine mandate" in tweet_strings[i].lower():
    filtered_tweets.append(tweet_strings[i])
  else:
    pass

In [128]:
len(filtered_tweets)

3

In [129]:
filtered_tweets

['Oklahoma National Guard rejects Pentagon’s coronavirus vaccine mandate https://t.co/aEDXoubNzZ',
 'RT @Bencjacobs: Per the exit polls, the Virginia electorate tonight supported both vaccine mandates and Confederate monuments https://t.co/…',
 "Biden's new vaccine mandate is so oppressive, tyrannical, dictatorial &amp; fascistic that you don't even have to get the vaccine under it and can just get tested. This is Hitler, Stalin, Mao and Pol Pot combined, to me."]

In [18]:
twitter_data_df

,News Twitter Handles,steve bannon,ahmaud arbery,cop26 climate,kyle rittenhouse,homer plessy,capitol riot,vaccine mandate,plane crash,2 high,central park,booster shot,sarah sander,restraining order,missing 5-year-old,declares state,national guard,24 year,u.s. capitol,d.c.-area forecast,found safe,bank robber,rpd man,amber alert,black autistic,teacher found,'hang mike,coronavirus oregon,water level,university chicago,snow accumulation,murkowski announces,fog expected,isabella kalua,gov bill,young wheeling,tracking saturday,ciattarelli concedes,andrew cuomo,new covid,john deere,2 adult,robert califf,elizabeth holmes,school board,golden state,'dolphin tale,madison cawthorn,chris christie,joe biden,cheney rip,hit-and-run outside,grocery store,subway track,pleads guilty,kamala harris,myanmar court,missouri man,xi jinping,rental assistance,ally possible,veteran day,case 83
0,@KyleKulinski,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,@krystalball,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,@dpakman,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,@cornchipsss,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,@progressvoice,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,@EmmaVigeland,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,@AnaKasparian,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,@SamSeder,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,@cenkuygur,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,@johniadarola,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Non

getting tweets before 1453806442646052864
getting tweets before 1445121290487091215
getting tweets before 1437921985326157828
getting tweets before 1427404655966003199
getting tweets before 1412547059912261631
getting tweets before 1405608047339360261
getting tweets before 1392589022766276612
getting tweets before 1371539282545029122
getting tweets before 1354520055669780490
getting tweets before 1351592104724672511
getting tweets before 1341537303743909888
getting tweets before 1337167385271410688
getting tweets before 1330521789840166912
getting tweets before 1325254662010904581
getting tweets before 1321267641735122943
getting tweets before 1319446522610810880
getting tweets before 1318388884003315711


In [21]:
#step 1: web scrape the sources from those websites and sample from the list of sources. (Add independent journalists to both lists)

In [22]:
#search the names of the news sources on twitter, make a dataframe containing the 

In [23]:
#need large list of top right leaning and left leaning sources for comparison

In [24]:
#step 2: categorize whether a news outlet is left leaning or right leaning using recommended users, followings, and followers 

In [25]:
#step 3: After classification, create a dataframe containing the classified left leaning and right leaning outlets + news topics 

In [26]:
#step 4: visualize the graph of relations

In [27]:
#optional: can webscrape fresh topics from internet website